# API City prices for living cost from numbeo.com

In [2]:
# import libraries
import requests
import os
import pandas as pd
import json
import sql_functions as sf

In [3]:
# To be able to pull the API key from the .env, we need to import load_dotenv

from dotenv import load_dotenv
load_dotenv()

True

Info from numbeo.com

yearLastUpdate': 2023

Selection list of items from numbeo.com for day_cost_bucket:

+ 'item_id': 1, 'item_name': 'Meal, Inexpensive Restaurant, Restaurants'
+ 'item_id': 2, 'item_name': 'Meal for 2 People, Mid-range Restaurant, Three-course, Restaurants'
+ 'item_id': 3, 'item_name': 'McMeal at McDonalds (or Equivalent Combo Meal), Restaurants'
+ 'item_id': 4, 'item_name': 'Domestic Beer (0.5 liter draught), Restaurants'
+ 'item_id': 6, 'item_name': 'Coke/Pepsi (0.33 liter bottle), Restaurants'
+ 'item_id': 7, 'item_name': 'Water (0.33 liter bottle), Restaurants'
+ 'item_id': 114, 'item_name': 'Cappuccino (regular), Restaurants'
+ 'item_id': 13, 'item_name': 'Water (1.5 liter bottle), Markets'
+ 'item_id': 18, 'item_name': 'One-way Ticket (Local Transport), Transportation'
+ 'item_id': 108, 'item_name': 'Taxi 1km (Normal Tariff), Transportation'


Our day_cost_bucket:

+ 'item_id': 1, 'item_name': 'Meal, Inexpensive Restaurant, Restaurants'
+ 'item_id': 3, 'item_name': 'McMeal at McDonalds (or Equivalent Combo Meal), Restaurants'
+ 'item_id': 4, 'item_name': 'Domestic Beer (0.5 liter draught), Restaurants'
+ 'item_id': 7, 'item_name': 'Water (0.33 liter bottle), Restaurants'
+ 'item_id': 114, 'item_name': 'Cappuccino (regular), Restaurants'
+ 'item_id': 13, 'item_name': 'Water (1.5 liter bottle), Markets'
+ 'item_id': 18, 'item_name': 'One-way Ticket (Local Transport), Transportation'
+ 'item_id': 108, 'item_name': 'Taxi 1km (Normal Tariff), Transportation'








In [8]:
#Load city list from data/cities_numbeo excel file
city_list = pd.read_excel('data/Cities_numbeo.xlsx')


In [10]:
city_list.head()

,City
0,"Aachen, Germany"
1,"Aalborg, Denmark"
2,"Aberdeen, United Kingdom"
3,"Abidjan, Ivory Coast"
4,"Abu Dhabi, United Arab Emirates"


In [16]:
#Split the list in three columns on comma ' , ' 
city_list[['city', 'country', 'country_2']] = city_list['City'].str.split(', ', expand=True)

In [17]:
city_list.head()

,City,city,country,country_2
0,"Aachen, Germany",Aachen,Germany,None
1,"Aalborg, Denmark",Aalborg,Denmark,None
2,"Aberdeen, United Kingdom",Aberdeen,United Kingdom,None
3,"Abidjan, Ivory Coast",Abidjan,Ivory Coast,None
4,"Abu Dhabi, United Arab Emirates",Abu Dhabi,United Arab Emirates,None


In [18]:
city_list.loc[city_list['country_2'].notnull(), 'country'] = city_list['country_2']

In [19]:
city_list.head()

,City,city,country,country_2
0,"Aachen, Germany",Aachen,Germany,None
1,"Aalborg, Denmark",Aalborg,Denmark,None
2,"Aberdeen, United Kingdom",Aberdeen,United Kingdom,None
3,"Abidjan, Ivory Coast",Abidjan,Ivory Coast,None
4,"Abu Dhabi, United Arab Emirates",Abu Dhabi,United Arab Emirates,None


In [20]:
# Check the result
city_list['country'].unique()

array(['Germany', 'Denmark', 'United Kingdom', 'Ivory Coast',
       'United Arab Emirates', 'Ghana', 'Saudi Arabia', 'Turkey',
       'Ethiopia', 'Australia', 'Morocco', 'India', 'United States',
       'Egypt', 'Algeria', 'Spain', 'Kazakhstan', 'Jordan', 'Netherlands',
       'Belgium', 'Paraguay', 'Greece', 'New Zealand', 'Portugal', 'Iraq',
       'Philippines', 'Azerbaijan', 'Indonesia', 'Thailand',
       'Bosnia And Herzegovina', 'Barbados', 'Italy', 'Colombia',
       'Canada', 'Switzerland', 'Georgia', 'China', 'Lebanon', 'Serbia',
       'Brazil', 'Norway', 'Kyrgyzstan', 'North Macedonia', 'France',
       'Romania', 'Slovakia', 'Czech Republic', 'Hungary', 'Montenegro',
       'Argentina', 'Bulgaria', 'Mexico', 'South Africa', 'Venezuela',
       'Moldova', 'Bangladesh', 'Sri Lanka', 'Ireland', 'Ecuador',
       'Vietnam', 'Senegal', 'Syria', 'Tanzania', 'Ukraine', 'Qatar',
       'Cameroon', 'Isle Of Man', 'Croatia', 'Finland', 'Botswana',
       'Poland', 'Cayman Islands',

In [21]:
# Drop the column City and country_2
city_list.drop(['City', 'country_2'], axis = 1, inplace = True)

In [22]:
city_list.head()

,city,country
0,Aachen,Germany
1,Aalborg,Denmark
2,Aberdeen,United Kingdom
3,Abidjan,Ivory Coast
4,Abu Dhabi,United Arab Emirates


In [23]:
city = list(city_list['city'])

In [24]:
city

['Aachen',
 'Aalborg',
 'Aberdeen',
 'Abidjan',
 'Abu Dhabi',
 'Accra',
 'Ad Dammam',
 'Adana',
 'Addis Ababa',
 'Adelaide',
 'Agadir',
 'Ahmedabad',
 'Ajman',
 'Al Khobar',
 'Alanya',
 'Albuquerque',
 'Alexandria',
 'Algiers',
 'Alicante',
 'Almaty',
 'Amman',
 'Amsterdam',
 'Ankara',
 'Antalya',
 'Antwerp',
 'Arhus',
 'Astana (Nur-Sultan)',
 'Asuncion',
 'Athens',
 'Atlanta',
 'Auckland',
 'Augsburg',
 'Austin',
 'Aveiro',
 'Baghdad',
 'Baguio',
 'Baku',
 'Bali',
 'Bandung',
 'Bangalore',
 'Bangkok',
 'Banja Luka',
 'Barbados',
 'Barcelona',
 'Bari',
 'Barranquilla',
 'Barrie',
 'Basel',
 'Batumi',
 'Beijing',
 'Beirut',
 'Belfast',
 'Belgrade',
 'Belo Horizonte',
 'Bergamo',
 'Bergen',
 'Berlin',
 'Bern',
 'Bhubaneswar',
 'Bilbao',
 'Birmingham',
 'Bishkek',
 'Bitola',
 'Bogota',
 'Boise',
 'Bologna',
 'Bonn',
 'Bordeaux',
 'Boston',
 'Bournemouth',
 'Braga',
 'Brampton',
 'Brasilia',
 'Brasov',
 'Bratislava',
 'Bremen',
 'Brescia',
 'Brighton',
 'Brisbane',
 'Bristol',
 'Brno',
 'B

In [25]:
# Make a country list for API query
country = list(city_list['country'])

In [26]:
country

['Germany',
 'Denmark',
 'United Kingdom',
 'Ivory Coast',
 'United Arab Emirates',
 'Ghana',
 'Saudi Arabia',
 'Turkey',
 'Ethiopia',
 'Australia',
 'Morocco',
 'India',
 'United Arab Emirates',
 'Saudi Arabia',
 'Turkey',
 'United States',
 'Egypt',
 'Algeria',
 'Spain',
 'Kazakhstan',
 'Jordan',
 'Netherlands',
 'Turkey',
 'Turkey',
 'Belgium',
 'Denmark',
 'Kazakhstan',
 'Paraguay',
 'Greece',
 'United States',
 'New Zealand',
 'Germany',
 'United States',
 'Portugal',
 'Iraq',
 'Philippines',
 'Azerbaijan',
 'Indonesia',
 'Indonesia',
 'India',
 'Thailand',
 'Bosnia And Herzegovina',
 'Barbados',
 'Spain',
 'Italy',
 'Colombia',
 'Canada',
 'Switzerland',
 'Georgia',
 'China',
 'Lebanon',
 'United Kingdom',
 'Serbia',
 'Brazil',
 'Italy',
 'Norway',
 'Germany',
 'Switzerland',
 'India',
 'Spain',
 'United Kingdom',
 'Kyrgyzstan',
 'North Macedonia',
 'Colombia',
 'United States',
 'Italy',
 'Germany',
 'France',
 'United States',
 'United Kingdom',
 'Portugal',
 'Canada',
 'Brazil

In [27]:
#Create a dictionary city, country for API query
city_count_dict = dict(zip(city, country))

In [28]:
city_count_dict

{'Aachen': 'Germany',
 'Aalborg': 'Denmark',
 'Aberdeen': 'United Kingdom',
 'Abidjan': 'Ivory Coast',
 'Abu Dhabi': 'United Arab Emirates',
 'Accra': 'Ghana',
 'Ad Dammam': 'Saudi Arabia',
 'Adana': 'Turkey',
 'Addis Ababa': 'Ethiopia',
 'Adelaide': 'Australia',
 'Agadir': 'Morocco',
 'Ahmedabad': 'India',
 'Ajman': 'United Arab Emirates',
 'Al Khobar': 'Saudi Arabia',
 'Alanya': 'Turkey',
 'Albuquerque': 'United States',
 'Alexandria': 'Egypt',
 'Algiers': 'Algeria',
 'Alicante': 'Spain',
 'Almaty': 'Kazakhstan',
 'Amman': 'Jordan',
 'Amsterdam': 'Netherlands',
 'Ankara': 'Turkey',
 'Antalya': 'Turkey',
 'Antwerp': 'Belgium',
 'Arhus': 'Denmark',
 'Astana (Nur-Sultan)': 'Kazakhstan',
 'Asuncion': 'Paraguay',
 'Athens': 'Greece',
 'Atlanta': 'United States',
 'Auckland': 'New Zealand',
 'Augsburg': 'Germany',
 'Austin': 'United States',
 'Aveiro': 'Portugal',
 'Baghdad': 'Iraq',
 'Baguio': 'Philippines',
 'Baku': 'Azerbaijan',
 'Bali': 'Indonesia',
 'Bandung': 'Indonesia',
 'Bangalore

In [29]:
# #API download from www.numbeo.com 
url = 'https://www.numbeo.com/api/city_prices'
api_key = 'zlueew978sczoi'
#country_list ' don`t need, we take the values for countries from country_list in cell above 
desired_item_ids = [1, 3, 4, 7, 114, 13, 18, 108]
currency = "EUR"

data_list = []

for city in city:
    # Send the HTTP GET request for each country with currency parameter
    response = requests.get(url, params={"api_key": api_key, "query": city, "currency": currency})
    
    # Check if the request was successful
    if response.status_code == 200:
        # Parse the JSON response
        data = response.json()
        
        # Access the "prices" list from the response
        prices = data.get("prices", [])
        
        # Access the country's land information
        city = city
        
        # Initialize a list to store the desired items for the country
        desired_items = []
        
        # Iterate over the prices and check if the item_id matches the desired values
        for price in prices:
            item_id = price.get("item_id")
            if item_id in desired_item_ids:
                desired_items.append(price)
        
        # Add the desired items, land information, and currency to the data list
        for item in desired_items:
            item["city"] = city
            item["currency"] = currency
        data_list.extend(desired_items)
    else:
        print(f"Request for {city} failed with status code:", response.status_code)

# Create a DataFrame from the data list
df = pd.DataFrame(data_list)

# Print the DataFrame
df

,data_points,item_id,lowest_price,average_price,highest_price,item_name,city,currency
0,23,1,8.000000,10.000000,13.990000,"Meal, Inexpensive Restaurant, Restaurants",Aachen,EUR
1,11,3,8.000000,9.000000,10.000000,McMeal at McDonalds (or Equivalent Combo Meal)...,Aachen,EUR
2,15,4,3.000000,3.500000,4.500000,"Domestic Beer (0.5 liter draught), Restaurants",Aachen,EUR
3,17,7,1.800000,2.225000,3.000000,"Water (0.33 liter bottle) , Restaurants",Aachen,EUR
4,12,13,0.250000,0.496250,1.000000,"Water (1.5 liter bottle), Markets",Aachen,EUR
...,...,...,...,...,...,...,...,...
4287,47,7,3.578252,4.576535,6.134146,"Water (0.33 liter bottle) , Restaurants",Zurich,EUR
4288,52,13,0.613415,1.332072,2.249187,"Water (1.5 liter bottle), Markets",Zurich,EUR
4289,45,18,3.169309,4.498374,6.338618,"One-way Ticket (Local Transport), Transportation",Zurich,EUR
4290,10,108,4.089431,5.111788,5.111788,"Taxi 1km (Normal Tariff), Transportation",Zurich,EUR


In [ ]:
#4292 rows and 8 columns

In [30]:
df.head(15)

,data_points,item_id,lowest_price,average_price,highest_price,item_name,city,currency
0,23,1,8.000000,10.000000,13.990000,"Meal, Inexpensive Restaurant, Restaurants",Aachen,EUR
1,11,3,8.000000,9.000000,10.000000,McMeal at McDonalds (or Equivalent Combo Meal)...,Aachen,EUR
2,15,4,3.000000,3.500000,4.500000,"Domestic Beer (0.5 liter draught), Restaurants",Aachen,EUR
3,17,7,1.800000,2.225000,3.000000,"Water (0.33 liter bottle) , Restaurants",Aachen,EUR
4,12,13,0.250000,0.496250,1.000000,"Water (1.5 liter bottle), Markets",Aachen,EUR
5,10,18,2.800000,2.950000,3.400000,"One-way Ticket (Local Transport), Transportation",Aachen,EUR
6,4,108,1.800000,1.950000,2.200000,"Taxi 1km (Normal Tariff), Transportation",Aachen,EUR
7,21,114,2.267858,2.885714,3.500000,"Cappuccino (regular), Restaurants",Aachen,EUR
8,11,1,12.755426,17.454794,32.089968,"Meal, Inexpensive Restaurant, Restaurants",Aalborg,EUR
9,5,3,10.741412,13.158229,14.769441,McMeal at McDonalds (or Equivalent Combo Meal)...,Aalborg,EUR


In [31]:
#Drop column data_points, item_id, lowest_price and highest_price
df_new = df.drop(["data_points", 'lowest_price', 'highest_price', 'currency'], axis=1, inplace=True)

In [35]:
#Add the column country
df['country'] = df['city'].map(city_count_dict)

In [36]:
df.head()

,item_id,average_price,item_name,city,country
0,1,10.00000,"Meal, Inexpensive Restaurant, Restaurants",Aachen,Germany
1,3,9.00000,McMeal at McDonalds (or Equivalent Combo Meal)...,Aachen,Germany
2,4,3.50000,"Domestic Beer (0.5 liter draught), Restaurants",Aachen,Germany
3,7,2.22500,"Water (0.33 liter bottle) , Restaurants",Aachen,Germany
4,13,0.49625,"Water (1.5 liter bottle), Markets",Aachen,Germany


In [37]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4292 entries, 0 to 4291
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   item_id        4292 non-null   int64  
 1   average_price  4286 non-null   float64
 2   item_name      4292 non-null   object 
 3   city           4292 non-null   object 
 4   country        4292 non-null   object 
dtypes: float64(1), int64(1), object(3)
memory usage: 167.8+ KB


## Push the DataFrame df to sql

In [38]:
#Define schema for DBeaver
schema = 'capstone_travel_index'
engine = sf.get_engine()

In [40]:
#Import dataset as table to DBeaver, rename df_clean to country_avg_price
table_name = 'city_living_prices'
if engine!=None:
    try:
        df.to_sql(name=table_name, # Name of SQL table
                        con=engine, # Engine or connection
                        if_exists='replace', # Drop the table before inserting new values 
                        schema=schema, # Use schmea that was defined earlier
                        index=False, # Write DataFrame index as a column
                        chunksize=5000, # Specify the number of rows in each batch to be written at a time
                        method='multi') # Pass multiple values in a single INSERT clause
        print(f"The {table_name} table was imported successfully.")
    # Error handling
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        engine = None

The city_living_prices table was imported successfully.


In [ ]:
#The city_living_prices table was imported successfully.